In [2]:
import xml
import xml.sax

In [10]:
class NodeHandler(xml.sax.ContentHandler):
    def __init__(self):
        self.CurrentData = ""
        self.lat = ""
        self.long = ""
        self.features = ""
        self.item_counter = 0
        
    def startElement(self, tag, attributes):
        self.CurrentData = tag
        
        if tag == "node" and self.item_counter < 100:
            self.item_counter += 1
            print("Node {0}: lat - {1}, long - {2}".format(
                attributes["id"],
                attributes["lat"],
                attributes["lon"]
                ))
    
    def endElement(self, tag):
        pass
    
    def characters(self, content):
        pass
    

In [11]:
parser = xml.sax.make_parser()
# parser.setFeature(xml.sax.handler.feature_namespaces, 0)
Handler = NodeHandler()
parser.setContentHandler(Handler)
parser.parse("sydney.osm 2")

Node 324883: lat - -33.91767, long - 151.18883
Node 324884: lat - -33.91085, long - 151.19492
Node 624529: lat - -33.89763, long - 151.22774
Node 624532: lat - -33.89724, long - 151.2289
Node 624534: lat - -33.89731, long - 151.22797
Node 624535: lat - -33.89693, long - 151.22981
Node 624536: lat - -33.89621, long - 151.23079
Node 624537: lat - -33.89533, long - 151.23147
Node 624539: lat - -33.89463, long - 151.23222
Node 624540: lat - -33.89434, long - 151.23275
Node 624541: lat - -33.89404, long - 151.23399
Node 624542: lat - -33.89408, long - 151.23529
Node 624543: lat - -33.8972, long - 151.22664
Node 624544: lat - -33.89825, long - 151.22802
Node 624546: lat - -33.90009, long - 151.22774
Node 624547: lat - -33.89895, long - 151.2279
Node 624549: lat - -33.89922, long - 151.22781
Node 624840: lat - -33.8886, long - 151.19883
Node 624843: lat - -33.88994, long - 151.2079
Node 624845: lat - -33.89138, long - 151.21166
Node 624846: lat - -33.89176, long - 151.21446
Node 624853: lat -

# Pulling out lat and long data for nodes 

In [1]:
# Can we store all of the lat and long information?
from xml.etree import ElementTree

counter = 0
with open('sydney.osm 2', 'rt') as f:
    tree = ElementTree.parse(f)
    
node_location = dict()
    
for node in tree.iter():
    # two kinds of nodes here - xml node and our point nodes
    if node.tag == "node":
        node_info = {
            "lat": node.attrib["lat"],
            "lon": node.attrib["lon"],
            }
        node_location[node.attrib["id"]] = node_info

In [2]:
counter = 0
for node in node_location:
    if counter < 100:
        print(node)
        counter += 1

3937343779
2355571594
3429583178
4292255199
2689506166
2052452153
4030433257
1856288779
1024370651
1153939777
1782392661
3203965163
4157488127
4246224417
1582092192
1190215004
774465516
4097655520
1869430227
2274973310
4100881837
81118263
1857106510
4106943159
2121992411
3996652522
2644423503
1832476531
4192898782
1835296082
2526164161
2822302973
1884919116
1131889515
361307256
17604321
288788017
3919297563
612032412
1215639455
252169095
1141657643
1844480927
3766348471
1695379615
2896460892
2820152558
92597705
913079168
4054113975
2139776894
1239442425
2334059840
599454616
1152070305
2980725968
2757776529
3240555687
1611285808
1836547351
2117157082
1222946273
616580876
1033533392
4047651656
245844854
2524023913
1827326780
3393287520
1247828858
3815456508
1128844549
3703981789
18260719
1865350042
1299461267
2505500685
1875310669
443617362
742630978
577455970
616051070
3870563403
1867764925
1838908860
1202465252
1099233400
2978564231
1017843287
1912764945
2354546844
1146610899
458446698

In [3]:
len(node_location)

963833

# Pulling out ways

In [4]:
with open('sydney.osm 2', 'rt') as f:
    tree = ElementTree.parse(f)
    
way_data = dict()
    
for node in tree.iter():
    # two kinds of nodes here - xml node and our point nodes
    if node.tag == "way":
        reference_node = node
        for subnode in node:
            print(subnode.tag)
        break
        #way_info = {
        #    "name": node.attrib["name"]
        #    "lat": node.attrib["lat"],
        #    "lon": node.attrib["lon"],
        #    }
        #node_location[node.attrib["id"]] = node_info

nd
nd
tag
tag
tag


In [5]:
for subnode in reference_node:
    print(subnode.attrib.keys())
    print(subnode.attrib['ref'])


dict_keys(['ref'])
1805891935
dict_keys(['ref'])
1805891958
dict_keys(['v', 'k'])


KeyError: 'ref'

In [7]:
# First prototype...

with open('sydney.osm 2', 'rt') as f:
    tree = ElementTree.parse(f)
    
way_data = dict()
counter = 0
    
for node in tree.iter():
    # two kinds of nodes here - xml node and our point nodes
    if counter == 100:
        break
        
    if node.tag == "way":
        counter += 1
        flag_to_keep = False
        polyline = list()
        way_info = dict()
        for subnode in node:
            # Try and pull out node locations
            try:
                node_ref = subnode.attrib['ref']
                location = {
                    "lat": node_location[node_ref]['lat'],
                    "lon": node_location[node_ref]['lon']
                }
                polyline.append(location)
            except:
                pass
            
            # Try and pull out street name
            try:
                key = subnode.attrib['k']
                if key == 'name':
                    way_info["name"] = subnode.attrib['v']
                # Unless we have the highway key, it's not a road.
                if key == 'highway':
                    flag_to_keep = True
            except:
                pass

        # Bundle this into the summary dict
        way_info["points"] = polyline
                    
        if flag_to_keep:
            way_data[node.attrib["id"]] = way_info
        #way_info = {
        #    "name": node.attrib["name"]
        #    "lat": node.attrib["lat"],
        #    "lon": node.attrib["lon"],
        #    }
        #node_location[node.attrib["id"]] = node_info

In [8]:
node_location["1559579795"]

{'lat': '-33.84138', 'lon': '151.20866'}

In [ ]:
print(way_data)

In [9]:
# First prototype...

with open('sydney.osm 2', 'rt') as f:
    tree = ElementTree.parse(f)
    
way_data = dict()
    
for node in tree.iter():
    # two kinds of nodes here - xml node and our point nodes
        
    if node.tag == "way":
        counter += 1
        flag_to_keep = False
        polyline = list()
        way_info = dict()
        for subnode in node:
            # Try and pull out node locations
            try:
                node_ref = subnode.attrib['ref']
                location = {
                    "lat": node_location[node_ref]['lat'],
                    "lon": node_location[node_ref]['lon']
                }
                polyline.append(location)
            except:
                pass
            
            # Try and pull out street name
            try:
                key = subnode.attrib['k']
                if key == 'name':
                    way_info["name"] = subnode.attrib['v']
                # Unless we have the highway key, it's not a road.
                if key == 'highway':
                    flag_to_keep = True
            except:
                pass

        # Bundle this into the summary dict
        way_info["points"] = polyline
                    
        if flag_to_keep:
            way_data[node.attrib["id"]] = way_info
        #way_info = {
        #    "name": node.attrib["name"]
        #    "lat": node.attrib["lat"],
        #    "lon": node.attrib["lon"],
        #    }
        #node_location[node.attrib["id"]] = node_info

In [10]:
len(way_data)


88167

In [11]:
# There might be streets in here that are outside of our assault data. 
# Therefore filter these ways...
import pandas as pd

In [12]:
geocrime_data = pd.read_csv("geocrimedata_sydney.csv", usecols=("bcsrgclat","bcsrgclng"))
min_lat = geocrime_data["bcsrgclat"].min()
max_lat = geocrime_data["bcsrgclat"].max() 
min_lon = geocrime_data["bcsrgclng"].min() 
max_lon = geocrime_data["bcsrgclng"].max()
print(min_lon)
print(max_lon)

151.1749663
151.23301


In [14]:
reduced_way_data = way_data.copy()
counter = 0
for way in list(way_data):
    counter += 1
    if counter % 1000 == 0:
        print(counter)
    keep_way = False
    for point in way_data[way]["points"]:
        good_lat = False
        good_lon = False
        if float(point["lat"]) > min_lat and float(point["lat"]) < max_lat:
            good_lat = True
        if float(point["lon"]) > min_lon and float(point["lon"]) < max_lon:
            good_lon = True
        if good_lat and good_lon:
            keep_way = True
            break
        
            
    if keep_way is False:
        del reduced_way_data[way]

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000


In [15]:
print(len(reduced_way_data))

8485


# Outputting datasets 

In [16]:
import json
with open("waydata.json", 'w') as f:
    json.dump(way_data, f, indent = 1)
with open("reducedwaydata.json", 'w') as f:
    json.dump(reduced_way_data, f, indent = 1)

In [52]:
with open("node_data.json", 'w') as f:
    json.dump(node_location, f, indent = 1)

# Non-assault Feature Selection

If we can map assaults to a road, it stands to reason that we should be able to map bars, police stations, etc.

In [ ]:
# Can we store all of the lat and long information?
from xml.etree import ElementTree

counter = 0
with open('sydney.osm 2', 'rt') as f:
    tree = ElementTree.parse(f)
    
node_location = dict()
    
for node in tree.iter():
    # two kinds of nodes here - xml node and our point nodes
    if node.tag == "node":
        node_info = {
            "lat": node.attrib["lat"],
            "lon": node.attrib["lon"],
            }
        node_location[node.attrib["id"]] = node_info

In [19]:
from xml.etree import ElementTree


with open('sydney.osm 2', 'rt') as f:
    tree = ElementTree.parse(f)
    
feature_location = dict()
# LIST THESE!
relevent_amenities = ("police", "atm", "place_of_worship", "hospital",
                     "restaurant", "drinking_water", "fast_food", "pub", "bar",
                     "parking", "taxi", "nightclub")

counter = 1
for node in tree.iter():
    # two kinds of nodes here - xml node and our point nodes
    if counter % 1000 == 0:
        print(counter)
        counter +=1
    if node.tag == "node":
        node_info = {
            "lat": node.attrib["lat"],
            "lon": node.attrib["lon"]
        }
        flag_to_keep = False
        polyline = list()
        way_info = dict()
        for subnode in node:
            # Try and pull out feature information
            try:
                key = subnode.attrib['k']
                if key == 'amenity':
                    if subnode.attrib['v'] in relevent_amenities:
                        node_info["feature"] = subnode.attrib['v']
                        flag_to_keep = True
                
                if key == 'highway':
                    if subnode.attrib['v'] == "bus_stop":
                        node_info["feature"] = "bus_stop"
                        flag_to_keep = True
                
                if key == "shop":
                    node_info["feature"] = "shop"
                    flag_to_keep = True
                
                if key == "tourism":
                    if subnode.attrib['v'] == "hostel":
                        node_info["feature"] = "hostel"
                        flag_to_keep = True

            except:
                pass
                    
        if flag_to_keep:
            feature_location[node.attrib["id"]] = node_info

In [20]:
counter = 0
for feature in feature_location:
    print(counter, feature_location[feature]['feature'], feature_location[feature]['lat'], feature_location[feature]['lon'])
    counter += 1
    if counter == 100:
        break

0 bus_stop -33.72255 151.00261
1 bus_stop -33.915 150.9022
2 bus_stop -33.87317 151.20844
3 bus_stop -33.82457 151.08788
4 fast_food -34.03484 151.10019
5 bus_stop -33.74453 151.00059
6 bus_stop -33.912 151.22337
7 shop -33.87487 151.20615
8 bus_stop -33.82755 151.01302
9 bus_stop -33.92878 150.96563
10 bus_stop -33.88794 150.97392
11 pub -33.86868 151.20442
12 bus_stop -33.76854 150.9915
13 bus_stop -33.8627 150.95699
14 bus_stop -33.80555 151.06995
15 shop -33.88704 151.2154
16 bus_stop -33.76238 150.99179
17 bus_stop -33.81901 151.01663
18 atm -34.01652 151.06654
19 bus_stop -33.77638 151.06795
20 bus_stop -33.84243 150.94967
21 bus_stop -33.94499 151.11348
22 parking -33.79319 150.89681
23 bus_stop -33.93003 151.03807
24 restaurant -33.84065 151.20673
25 bus_stop -33.80836 150.93912
26 bus_stop -33.76899 150.93223
27 shop -33.87061 151.21878
28 bus_stop -33.94109 151.11583
29 drinking_water -33.84059 151.21566
30 shop -33.88747 151.12233
31 hospital -33.82745 151.23179
32 place_of_

In [21]:
import json
with open("featuredata.json", 'w') as f:
    json.dump(feature_location, f, indent = 1)